<h1>Imports</h1>

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report

from imblearn.combine import SMOTETomek


In [ ]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [ ]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [ ]:
DATA_PATH = Path.cwd() / "../../datasets"

In [ ]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

In [ ]:
training_df = training_df.drop('pickup_time', 1)
training_df = training_df.drop('drop_time', 1)

<h1>Data Preprocessing</h1>

In [ ]:
features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','fare']]
labels = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
labels = labels.replace({'label':mapping})

In [ ]:
numerical_features = features.columns[features.dtypes != "object"].values

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(features, labels, test_size=0.3, shuffle=True, stratify= labels, random_state=RANDOM_SEED)

<h1>Data Cleaning</h1>

<h1>Feature Engineering</h1>

<h1>Model Training</h1>

In [ ]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])


## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [ ]:
preprocessed_features = preprocessor.fit_transform(X_train)
preprocessed_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [ ]:
preprocessed_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [ ]:
smk = SMOTETomek(random_state=42)
os_x_train, os_y_train = smk.fit_sample(preprocessed_features_data_frame,y_train)

In [ ]:
os_y_train.shape

In [ ]:
estimators = MultiOutputClassifier(
    estimator = RandomForestClassifier(random_state=10)
)

In [ ]:
full_pipeline = Pipeline([
    ('estimators', estimators)
])

full_pipeline

In [ ]:
estimator =  RandomForestClassifier(max_depth=2, random_state=0)

## Train the model
estimator .fit(os_x_train, os_y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (estimator.score(preprocessed_test_features_data_frame, y_eval)*100), "%")
y_pred = estimator.predict(preprocessed_test_features_data_frame)

In [ ]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

In [ ]:
print(classification_report(y_eval, y_pred))

In [ ]:
estimator.feature_importances_

<h1>Model Validation</h1>

In [ ]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

In [ ]:
test_probs = estimator.predict(test_set[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','fare']])

In [ ]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= test_probs

In [ ]:
submission_set.to_csv('../../submissions/model4/teamCluster_submission_01.csv', index=True)
print("Completed!")

In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()